In [3]:
%%capture
!pip install ijson
!pip install python-terrier
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])
import pandas as pd
import numpy as np



In [9]:
from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles = []
title_ids = []
nums = []
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    nums.append(num)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    nums.append(num)
topics = pd.DataFrame()
topics['qid'] = nums
topics['query'] = titles

qrels_path=("/workspace/src/evaluate_targers/qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

docnos = qrels.docno


In [7]:
import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100"

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            print('\n' + subdir)
            path = subdir + '/'+ file
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dph = pt.BatchRetrieve(index, wmodel="DPH")
            dph_rm3 = (pt.BatchRetrieve(indexref, wmodel="DPH") >> 
            pt.rewrite.RM3(indexref) >> 
            pt.BatchRetrieve(indexref, wmodel="DPH"))
            bm25_rm3 = (pt.BatchRetrieve(indexref, wmodel="BM25") >> 
            pt.rewrite.RM3(indexref) >> 
            pt.BatchRetrieve(indexref, wmodel="BM25"))
            tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
            bm25 = pt.BatchRetrieve(index, wmodel="BM25")
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM")
            dlm_rm3 = (pt.BatchRetrieve(indexref, wmodel="DirichletLM") >> 
            pt.rewrite.RM3(indexref) >> 
            pt.BatchRetrieve(indexref, wmodel="DirichletLM"))
            models = [dph, dph_rm3, tf_idf, bm25_rm3, bm25, dlm, dlm_rm3]
            print(pt.Experiment(models, topics, qrels, eval_metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"]))


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
                                                name  ndcg_cut_5     bpref  \
0                                            BR(DPH)    0.640618  0.627961   
1  Compose(Compose(BR(DPH), <pyterrier.rewrite.RM...    0.629165  0.639889   
2                                         BR(TF_IDF)    0.552754  0.626148   
3  Compose(Compose(BR(BM25), <pyterrier.rewrite.R...    0.583558  0.645344   
4                                           BR(BM25)    0.554767  0.626163   
5                                    BR(DirichletLM)    0.639554  0.599241   
6  Compose(Compose(BR(DirichletLM), <pyterrier.re...    0.514243  0.547045   

   ndcg_cut_25  
0     0.534502  
1     0.534129  
2     0.457499  
3     0.473546  
4     0.460291  
5     0.517360  
6     0.350469  


In [10]:
import os

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            print('\n' + subdir)
            for model in models:
                res= model.transform(topics)
                res_copy = res[res['docno'].isin(docnos)].copy()
                print(f'model : {model}, res: {pt.Utils.evaluate(res_copy,qrels,metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"])}')


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
model : BR(DPH), res: {'ndcg_cut_5': 0.6764085089131627, 'bpref': 0.6279613513936027, 'ndcg_cut_25': 0.6664826868151493}
model : Compose(Compose(BR(DPH), <pyterrier.rewrite.RM3 object at 0x7f6661585b90>), BR(DPH)), res: {'ndcg_cut_5': 0.6806878542050084, 'bpref': 0.6398893119563615, 'ndcg_cut_25': 0.6746344404708747}
model : BR(TF_IDF), res: {'ndcg_cut_5': 0.6118904950108662, 'bpref': 0.6261477227722415, 'ndcg_cut_25': 0.6332115060794034}
model : Compose(Compose(BR(BM25), <pyterrier.rewrite.RM3 object at 0x7f6661595a90>), BR(BM25)), res: {'ndcg_cut_5': 0.6448762370447452, 'bpref': 0.6453442081592141, 'ndcg_cut_25': 0.64542825542097}
model : BR(BM25), res: {'ndcg_cut_5': 0.6129710858217261, 'bpref': 0.6261632902519492, 'ndcg_cut_25': 0.6332690354359014}
model : BR(DirichletLM), res: {'ndcg_cut_5': 0.6629903601351183, 'bpref': 0.5992405823459444, 'ndcg_cut_25': 0.64174455658